In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split


task = 'abt_buy'

df1 = pd.read_csv('DATA/'+task+'/train.csv')
df2 = pd.read_csv('DATA/'+task+'/valid.csv')
df3 = pd.read_csv('DATA/'+task+'/test.csv')
clean = pd.concat([df1, df2, df3], ignore_index=True)
clean['id'] = range(len(clean))



df1_ = pd.read_csv('DATA/'+task+'/noisy_train.csv')
df2_ = pd.read_csv('DATA/'+task+'/noisy_valid.csv')
df3_ = pd.read_csv('DATA/'+task+'/noisy_test.csv')
dirty = pd.concat([df1_, df2_, df3_], ignore_index=True)
dirty['id'] = range(len(dirty))
dirty.drop(columns=['label','id'], inplace=True)



for col in dirty.columns:
    clean[col + '_noisy']=dirty[col] 





df = clean.copy()
X = df.drop(columns=['label'], inplace=False)
y = df['label']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_temp = X.iloc[train_index], X.iloc[test_index]
    y_train, y_temp = y.iloc[train_index], y.iloc[test_index]
    
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42, shuffle=True)
    
    train = pd.concat([y_train,X_train], axis=1)
    valid = pd.concat([y_val,X_val], axis=1)

    for col in train.columns:
        if 'noisy' in col:
            train.drop(columns=[col], inplace=True)
            valid.drop(columns=[col], inplace=True)


    TEST = pd.concat([y_test,X_test], axis=1)
    filtered_columns = [col for col in TEST.columns if col.startswith('left_') and not col.endswith('noisy')]
    new_columns = [col[5:] for col in filtered_columns]  


    if not os.path.exists('DATA/'+task+'_folded'): os.makedirs('DATA/'+task+'_folded')
    if not os.path.exists('DATA/'+task+'_folded/'+str(fold+1)): os.makedirs('DATA/'+task+'_folded/'+str(fold+1))

    train.to_csv('DATA/'+task+'_folded/'+str(fold+1)+'/train.csv', index=False)
    valid.to_csv('DATA/'+task+'_folded/'+str(fold+1)+'/valid.csv', index=False)

    for frac in [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
        num_rows = int(frac* len(TEST))
        for i in range(10):
            test = TEST.copy()
            random_indices = np.random.choice(test.index, num_rows, replace=False)
            for row in new_columns:
                test.loc[random_indices, 'left_'+row] = test.loc[random_indices, 'left_'+row+'_noisy']
                test.loc[random_indices, 'right_'+row] = test.loc[random_indices, 'right_'+row+'_noisy']

                test.drop(columns=['left_'+row+'_noisy'], inplace=True)
                test.drop(columns=['right_'+row+'_noisy'], inplace=True)

            (train, valid, test)
            frac2 = str(int(frac*100))
            if len(frac2) == 1: frac2 = frac2 +'0'

            test.to_csv('DATA/'+task+'_folded/'+str(fold+1)+'/test_'+frac2+'_'+str(i+1)+'.csv', index=False)

